# Introduction to Azure Language API for Security (PII, classification etc.)


https://learn.microsoft.com/en-us/azure/ai-services/language-service/personally-identifiable-information/overview

#### Follow [README](https://github.com/tirtho/open-ai/blob/main/README.md) and perform setup before running the notebooks

#### Load the API key and relevant Python libaries.

In [ ]:
# Run the first time
# pip install azure-ai-textanalytics==5.2.0
# pip install azure-ai-language-conversations --pre

In [30]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient
from azure.ai.textanalytics import PiiEntityDomain
from azure.ai.textanalytics import PiiEntityCategory

ai_key = os.environ.get('AZURE_AI_KEY')
ai_endpoint = os.environ.get('AZURE_AI_ENDPOINT')

aiServiceCredential = AzureKeyCredential(ai_key)
textAnalyticsClient = TextAnalyticsClient(
                            endpoint=ai_endpoint,
                            credential=aiServiceCredential
                        )

## Call PII
https://learn.microsoft.com/en-us/azure/ai-services/language-service/personally-identifiable-information/how-to-call

In [31]:
# Create a function to detect & display PII
categories = [
    "USIndividualTaxpayerIdentification",
    "USSocialSecurityNumber"
]

def getPII (textArray):
    response = textAnalyticsClient.recognize_pii_entities(
                documents=textArray,
                #categories_filter=categories,
                language="en"
            )
    result = [doc for doc in response if not doc.is_error]
    for doc in result:
        print("Entity Redacted Text: {}".format(doc.redacted_text))
        for entity in doc.entities:
            print("Entity: {}".format(entity))
            print("Entity Text: {}".format(entity.text))
            print("\tCategory: {}".format(entity.category))
            print("\tConfidence Score: {}".format(entity.confidence_score))
            print("\tOffset: {}".format(entity.offset))
            print("\tLength: {}".format(entity.length))

In [32]:
# Examples to detect SSN
# Find SSNs for test from this website below:
# https://en.wikipedia.org/wiki/Social_Security_number
# Valid SSNs are 123-45-1234, 010-01-0001
# SSNs for advertisement (is invalid) is 078-05-1120
# Adding these to the list for test.
# For each SSN, adding with and without context and without the format
validSSNsForPIIDetection = [
"SSN# 123-45-1234",
"123-45-1234",
"123451234"
]

getPII(validSSNsForPIIDetection)

Entity Redacted Text: SSN# ***********
Entity: {'text': '123-45-1234', 'category': 'USSocialSecurityNumber', 'subcategory': None, 'length': 11, 'offset': 5, 'confidence_score': 0.85}
Entity Text: 123-45-1234
	Category: USSocialSecurityNumber
	Confidence Score: 0.85
	Offset: 5
	Length: 11
Entity Redacted Text: ***********
Entity: {'text': '123-45-1234', 'category': 'USSocialSecurityNumber', 'subcategory': None, 'length': 11, 'offset': 0, 'confidence_score': 0.65}
Entity Text: 123-45-1234
	Category: USSocialSecurityNumber
	Confidence Score: 0.65
	Offset: 0
	Length: 11
Entity Redacted Text: *********
Entity: {'text': '123451234', 'category': 'PhoneNumber', 'subcategory': None, 'length': 9, 'offset': 0, 'confidence_score': 0.8}
Entity Text: 123451234
	Category: PhoneNumber
	Confidence Score: 0.8
	Offset: 0
	Length: 9


In [35]:
validSSNsForPIIDetection = [
"SSN# 010-01-0001",
"010-01-0001",
"010010001"
]
getPII(validSSNsForPIIDetection)

Entity Redacted Text: SSN# ***********
Entity: {'text': '010-01-0001', 'category': 'USSocialSecurityNumber', 'subcategory': None, 'length': 11, 'offset': 5, 'confidence_score': 0.85}
Entity Text: 010-01-0001
	Category: USSocialSecurityNumber
	Confidence Score: 0.85
	Offset: 5
	Length: 11
Entity Redacted Text: ***********
Entity: {'text': '010-01-0001', 'category': 'USSocialSecurityNumber', 'subcategory': None, 'length': 11, 'offset': 0, 'confidence_score': 0.65}
Entity Text: 010-01-0001
	Category: USSocialSecurityNumber
	Confidence Score: 0.65
	Offset: 0
	Length: 11
Entity Redacted Text: *********
Entity: {'text': '010010001', 'category': 'PhoneNumber', 'subcategory': None, 'length': 9, 'offset': 0, 'confidence_score': 0.8}
Entity Text: 010010001
	Category: PhoneNumber
	Confidence Score: 0.8
	Offset: 0
	Length: 9


In [37]:
# Invalid SSN. This is the famous one used for an ad in 1938
invalidSSNsForPIIDetection = [
"SSN# 078-05-1120",
"078-05-1120",
"078051120"
]
getPII(invalidSSNsForPIIDetection)

Entity Redacted Text: SSN# 078-05-1120
Entity Redacted Text: 078-05-1120
Entity Redacted Text: *********
Entity: {'text': '078051120', 'category': 'PhoneNumber', 'subcategory': None, 'length': 9, 'offset': 0, 'confidence_score': 0.8}
Entity Text: 078051120
	Category: PhoneNumber
	Confidence Score: 0.8
	Offset: 0
	Length: 9


In [39]:
# Invalid SSN. This is a monotonicaly increasing integer series.
invalidSSNsForPIIDetection = [
"SSN# 123-45-6789",
"123-45-6789",
"123456789"
]
getPII(invalidSSNsForPIIDetection)

Entity Redacted Text: SSN# 123-45-6789
Entity Redacted Text: 123-45-6789
Entity Redacted Text: *********
Entity: {'text': '123456789', 'category': 'PhoneNumber', 'subcategory': None, 'length': 9, 'offset': 0, 'confidence_score': 0.8}
Entity Text: 123456789
	Category: PhoneNumber
	Confidence Score: 0.8
	Offset: 0
	Length: 9


In [26]:
textAnalyticsClient.close()

In [41]:
pip install azure-ai-language-conversations


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\tibarar\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/122.0 kB ? eta -:--:--
   ------ -------------------------------- 20.5/122.0 kB 640.0 kB/s eta 0:00:01
   ------------- ------------------------- 41.0/122.0 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 122.0/122.0 kB 1.0 MB/s eta 0:00:00


## Call PII for Conversations
https://learn.microsoft.com/en-us/azure/ai-services/language-service/personally-identifiable-information/how-to-call-for-conversations?tabs=client-libraries